# セットアップ

jupyterからPostgreSQLに接続するための設定が以下のセルです。

In [1]:
%load_ext sql
import os

pgconfig = {
    "host": "db",
    "port": "5432",
    "database": os.environ["POSTGRES_DB"],
    "user": os.environ["POSTGRES_USER"],
    "password": os.environ["POSTGRES_PASSWORD"],
}
dsl = "postgresql://{user}:{password}@{host}:{port}/{database}".format(**pgconfig)

os.environ["DATABASE_URL"] = dsl
%sql

In [5]:
%sql select schemaname, tablename, tableowner from pg_tables;

Running query in 'postgresql://postgres:***@db:5432/postgres'

86 rows affected.

schemaname,tablename,tableowner
pg_catalog,pg_statistic,postgres
pg_catalog,pg_type,postgres
pg_catalog,pg_foreign_table,postgres
pg_catalog,pg_authid,postgres
pg_catalog,pg_statistic_ext_data,postgres
public,access_log_wide,postgres
public,access_log_dyn,postgres
public,search_log_dyn,postgres
public,monthly_sales,postgres
public,yearly_orders,postgres


In [6]:
%sql select * from pg_database ;

Running query in 'postgresql://postgres:***@db:5432/postgres'

3 rows affected.

oid,datname,datdba,encoding,datlocprovider,datistemplate,datallowconn,datconnlimit,datfrozenxid,datminmxid,dattablespace,datcollate,datctype,daticulocale,datcollversion,datacl
5,postgres,10,6,c,False,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.36,None
1,template1,10,6,c,True,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.36,"{=c/postgres,postgres=CTc/postgres}"
4,template0,10,6,c,True,False,-1,717,1,1663,en_US.utf8,en_US.utf8,None,None,"{=c/postgres,postgres=CTc/postgres}"


In [3]:
%%sql
select * from access_log limit 10;

Running query in 'postgresql://postgres:***@db:5432/postgres'

10 rows affected.

request_time,request_method,request_path,customer_id,search_hit,referer
2012-01-01 00:00:02,POST,/search,59856,38,None
2012-01-01 00:02:45,GET,/items/218,59856,None,/search
2012-01-01 00:04:07,POST,/search,59856,25,/items/218
2012-01-01 00:01:43,GET,/,79868,None,http://google.co.jp
2012-01-01 00:03:20,POST,/search,84526,26,None
2012-01-01 00:05:00,GET,/,70345,None,None
2012-01-01 00:05:10,POST,/search,70345,38,/
2012-01-01 00:05:52,POST,/search,70345,43,/search
2012-01-01 00:07:11,POST,/search,70345,22,/search
2012-01-01 00:06:38,GET,/,95994,None,None


In [10]:
%sql rollback;

Running query in 'postgresql://postgres:***@db:5432/postgres'

In [7]:
%sql select * from access_log_wide;

Running query in 'postgresql://postgres:***@db:5432/postgres'

5 rows affected.

request_time,request_month,request_method,request_path,customer_id,customer_name,customer_age,customer_birthday,search_hit,referer
2014-12-29 03:34:01,2014-12,GET,/,54115,森一郎,26,1989-05-16,None,https://google.co.jp
2014-12-29 03:34:05,2014-12,GET,/search,104097,丸山冴子,19,1996-08-01,154,/
2014-12-29 03:34:34,2014-12,GET,/,324,陣内太郎,37,1978-11-21,None,https://google.co.jp
2014-12-29 03:34:40,2014-12,GET,/,104097,丸山冴子,19,1996-08-01,None,/search
2014-12-29 03:35:05,2014-12,GET,/items/531415,93292,綾小路涼,41,1974-02-21,None,/search


In [9]:
%%sql
select
 request_time, customer_id
from
  access_log_wide
;

Running query in 'postgresql://postgres:***@db:5432/postgres'

5 rows affected.

request_time,customer_id
2014-12-29 03:34:01,54115
2014-12-29 03:34:05,104097
2014-12-29 03:34:34,324
2014-12-29 03:34:40,104097
2014-12-29 03:35:05,93292


In [12]:
%sql show search_path;

Running query in 'postgresql://postgres:***@db:5432/postgres'

1 rows affected.

search_path
"""$user"", public"
